In [1]:
#imports
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split


In [2]:
#read text data
df = pd.read_csv('processed_text/NLP_data_1.csv')
#verify data groups
df.groupby('Category').describe()

Code                                   
                      count mean  std  min  25%  50%  75%  max
Category                                                      
Blockchain           1572.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Cryptocurrency       1352.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Genetic engineering   957.0  2.0  0.0  2.0  2.0  2.0  2.0  2.0
Machine learning      648.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0
Nanotechnology       1489.0  4.0  0.0  4.0  4.0  4.0  4.0  4.0
Quantum computing    1606.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0
Robotics              910.0  6.0  0.0  6.0  6.0  6.0  6.0  6.0
Social engineering    895.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0
Space exploration    1308.0  8.0  0.0  8.0  8.0  8.0  8.0  8.0
Virtual reality       646.0  9.0  0.0  9.0  9.0  9.0  9.0  9.0

In [3]:
#train/test split

X_train,X_test,y_train,y_test = train_test_split(df['Text'],df['Code'])

In [4]:
y_train.value_counts()

5    1195
0    1184
4    1094
1    1030
8     981
2     729
6     682
7     681
3     489
9     472
Name: Code, dtype: int64

In [5]:
y_test.value_counts()

5    411
4    395
0    388
8    327
1    322
6    228
2    228
7    214
9    174
3    159
Name: Code, dtype: int64

In [8]:
test_data = pd.DataFrame()
test_data['x_test'] = X_test
test_data['y_test'] = y_test
#save test data for use later
test_data.to_csv('NLP_1_test_data.csv')
test_data

,x_test,y_test
10842,the virtual reality \nexperience should suppor...,9
10368,johnson presented a \nhope that the new intern...,8
10797,there are also minimum technical requirements ...,9
8023,• what causes some collegiate robotics competi...,6
588,"also, incentive features such as crypto\nasset...",0
...,...,...
7408,platform heterogeneity – the published technic...,5
1359,the impact of blockchain on the audit professi...,0
9726,"additionally, ac had a difficulty optimizing i...",8
7450,due to difficulties in quickly launching addit...,5


In [7]:
#BERT base (v4) preprocessing and encoder.
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [8]:
#NPL architechture consisting of an input layer with
#neurons equal to the BERT output length. A dropout layer with 10% attenuation,
#and a 10 neuron output layer.
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_encoder(preprocessed_text)

d = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
d = tf.keras.layers.Dense(10, activation='sigmoid', name='output')(d)

model = tf.keras.Model(inputs=[text_input], outputs = [d])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [9]:
#compile and fit
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
start = time()
history = model.fit(X_train,y_train,epochs=10)
train_time = (time()-start)

Epoch 1/10


C:\Users\Username_\.conda\envs\textML\lib\site-packages\tensorflow\python\util\dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


267/267 [==============================] - 148s 507ms/step - loss: 2.1296 - accuracy: 0.2435
Epoch 2/10
267/267 [==============================] - 137s 511ms/step - loss: 1.8728 - accuracy: 0.3706
Epoch 3/10
267/267 [==============================] - 136s 510ms/step - loss: 1.7191 - accuracy: 0.4343
Epoch 4/10
267/267 [==============================] - 136s 510ms/step - loss: 1.6074 - accuracy: 0.4808
Epoch 5/10
267/267 [==============================] - 136s 510ms/step - loss: 1.5285 - accuracy: 0.5145
Epoch 6/10
267/267 [==============================] - 136s 510ms/step - loss: 1.4683 - accuracy: 0.5221
Epoch 7/10
267/267 [==============================] - 136s 510ms/step - loss: 1.4177 - accuracy: 0.5487
Epoch 8/10
267/267 [==============================] - 135s 504ms/step - loss: 1.3745 - accuracy: 0.5564
Epoch 9/10
267/267 [==============================] - 133s 498ms/step - loss: 1.3484 - accuracy: 0.5592
Epoch 10/10
267/267 [==============================] - 133s 497ms/step - lo

In [10]:

eval = model.evaluate(X_test,y_test)

89/89 [==============================] - 46s 496ms/step - loss: 1.2435 - accuracy: 0.6174


In [18]:
# saves the evaluation, loss and acuracy into a csv file,
# saves the trained model as a keras .h5 file
# saves the model architecture summary as a .txt file
def save_model_data(model, model_evaluation, model_history, model_name):
    # initilize df1 and add evaluation results
    eval = pd.DataFrame(model_evaluation)
    eval = eval.T
    eval['epoch'] = 'evaluation'
    eval = eval.rename(columns = {0:'loss',1:'accuracy'})
    # initilize df2 and add .fit metrics
    history = pd.DataFrame(model_history.history)
    history['epoch'] = np.arange(len(history))
    history['epoch'] = history['epoch'].apply(lambda x: x + 1)
    # combine df1 and df2
    data = history.append(eval)
    # export to csv
    data.to_csv(str(model_name + '.csv'))
    # save the model
    model.save(model_name)
    # save model architecture, by user: sparklingdew 
    # https://stackoverflow.com/questions/45199047/how-to-save-model-summary-to-file-in-keras
    with open('modelsummary.txt', 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))

In [19]:
save_model_data(model,eval,history,'NLP_1')

In [15]:
print(train_time)

2.2751524527867635
